In [1]:
import sys, os
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

In [2]:
import data_utils.utils as du
import data_utils.ner as ner
from nerwindow import full_report, eval_performance
from nerwindow import WindowMLP

In [3]:
wv, word_to_num, num_to_word = ner.load_wv('data/vocab.txt',
                                           'data/wvec.txt')

tagnames = ["O", "B", "I", "L", "U"]
num_to_tag = dict(enumerate(tagnames))
tag_to_num = du.invert_dict(num_to_tag)

In [4]:
wsize = 5
hidden_dim = 100
reg = 0.001
alpha = 0.01
nepoch = 5
k = 5 

In [9]:
pad = (wsize - 1)/2

X_train, y_train, words_train = du.generateData('data/train_small', word_to_num, tag_to_num, wsize)
X_dev, y_dev, words_dev = du.generateData('data/dev_small', word_to_num, tag_to_num, wsize)

In [10]:
print X_train[:10]
print words_train[:10]

[[   1    1  201  534   28]
 [   1  201  534   28    5]
 [ 201  534   28    5  425]
 [ 534   28    5  425   42]
 [  28    5  425   42   10]
 [   5  425   42   10   86]
 [ 425   42   10   86  118]
 [  42   10   86  118 1412]
 [  10   86  118 1412   15]
 [  86  118 1412   15   20]]
[['<s>' '<s>' 'Excellent' 'Chinese' 'food']
 ['<s>' 'Excellent' 'Chinese' 'food' ',']
 ['Excellent' 'Chinese' 'food' ',' 'although']
 ['Chinese' 'food' ',' 'although' 'there']
 ['food' ',' 'although' 'there' 'was']
 [',' 'although' 'there' 'was' 'too']
 ['although' 'there' 'was' 'too' 'much']
 ['there' 'was' 'too' 'much' 'broccoli']
 ['was' 'too' 'much' 'broccoli' 'in']
 ['too' 'much' 'broccoli' 'in' 'my']]


In [7]:
def isCapital(x):
    if len(x) > 0:
        return x[0].isupper()
    return false

def capitalFeature(words):
    fvec = array([isCapital(ww) for ww in words]).astype(int)
    return fvec

cap_train = capitalFeature(words_train)
cap_dev = capitalFeature(words_dev)

In [8]:
print cap_train[:50]

[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 1 0 0 0 0]


In [ ]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.regularizers import l2

Xt = wv[X_train]
Xd = wv[X_dev]
X_t = Xt.reshape(Xt.shape[0], Xt.shape[1]*Xt.shape[2])
X_d = Xd.reshape(Xd.shape[0], Xd.shape[1]*Xd.shape[2])
Y_t = np_utils.to_categorical(y_train, 5)
Y_d = np_utils.to_categorical(y_dev, 5)
print X_t.shape
print Y_t.shape